In [1]:
import requests
import sqlite3
import pandas as pd
from smolagents import tool, Tool, CodeAgent, OpenAIServerModel, PromptTemplates
import os
from langchain_community.document_loaders import UnstructuredEPubLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import chromadb
from uuid import uuid4
from chromadb.utils import embedding_functions
from dotenv import load_dotenv
import matplotlib.pyplot as plt



Initialize model

In [2]:
result = load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

model_name = "gpt-4.1-mini"

agent_model = OpenAIServerModel(model_id=model_name, api_key = OPENAI_API_KEY)

Declare tools

In [3]:
class SQLiteTool(Tool):
#class mostly used for db 

   #need to give it a name
   name = "manga_database"

   #description
   description = """
   Performs sql queries on the manga table. Returns the result as an array of tuples.
      The table name is manga and has the following columns
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        genre TEXT,
        author TEXT,
        date_published TEXT,
        rating REAL
   """

   #arguments ( a dict)
   inputs = {
      "query": {
         "type": "string",
         "description": 'Valid SQL Statement'
      }
   }

   #Return type
   output_type = "any"
   

   def __init__(self, db_file):
      self.db_file = db_file 
      self.setup()

   #Part of the framework
   def setup(self):
      super().setup()
      self.conn = sqlite3.connect(self.db_file)

   def forward(self, query: str) -> str:
      try:
         cursor = self.conn.cursor() 
         rows = cursor.execute(query)
         return rows.fetchall()
      except Exception as e:
         print(f'Query exception: {e}')  

In [4]:
import matplotlib.pyplot as plt
import os
import uuid
from smolagents import Tool


class ChartTool(Tool):
    name = "chart_plotter"

    description = """
    Plots data into a chart using matplotlib.
    Accepts x and y values (as arrays) and creates either a line or bar chart.
    Returns the path to the saved chart image.

    Parameters:
      - x: array of x values
      - y: array of y values
      - chart_type: "line" or "bar" (optional, default: "line")
      - title: chart title (optional)
      - xlabel: x-axis label (optional)
      - ylabel: y-axis label (optional)
    """

    inputs = {
        "x": {
            "type": "array",
            "description": "Array of x-axis values"
        },
        "y": {
            "type": "array",
            "description": "Array of y-axis values"
        },
        "chart_type": {
            "type": "string",
            "description": "Type of chart to draw — 'line' or 'bar'",
            "nullable": True
        },
        "title": {
            "type": "string",
            "description": "Chart title",
            "nullable": True
        },
        "xlabel": {
            "type": "string",
            "description": "Label for the x-axis",
            "nullable": True
        },
        "ylabel": {
            "type": "string",
            "description": "Label for the y-axis",
            "nullable": True
        }
    }

    output_type = "string"

    def forward(self, x, y, chart_type="line", title="", xlabel="", ylabel=""):
        os.makedirs("charts", exist_ok=True)
        filename = f"charts/chart_{uuid.uuid4().hex[:8]}.png"
        plt.figure(figsize=(8, 5))

        try:
            if chart_type.lower() == "line":
                plt.plot(x, y, marker='o', linestyle='-', color='steelblue')
            elif chart_type.lower() == "bar":
                plt.bar(x, y, color='orange', edgecolor='black')
            else:
                raise ValueError("Unsupported chart type. Use 'line' or 'bar'.")

            plt.title(title or "Chart")
            plt.xlabel(xlabel or "X-axis")
            plt.ylabel(ylabel or "Y-axis")
            plt.grid(True, linestyle='--', alpha=0.6)
            plt.tight_layout()
            plt.savefig(filename)
            plt.close()

            return f"Chart saved at: {os.path.abspath(filename)}"

        except Exception as e:
            plt.close()
            return f"Chart generation failed: {e}"


In [5]:
tools = [ SQLiteTool(''), ChartTool()]


In [6]:
system_prompt = """You are a data visualization assistant with access to a manga database and charting tools.

WORKFLOW:
1. When asked to create a chart, ALWAYS query the database first using manga_database tool
2. Analyze the returned data (list of tuples)
3. Extract and transform the data into x and y lists
4. Call the chart_plotter tool with the prepared data

DATA TRANSFORMATION:
- Database queries return: [(value1_x, value1_y), (value2_x, value2_y), ...]
- You must convert this to: x=[value1_x, value2_x, ...], y=[value1_y, value2_y, ...]
- Example code: x = [row[0] for row in results], y = [row[1] for row in results]

DATABASE SCHEMA:
Table: manga
Columns: id, title, genre, author, date_published, rating

Always provide clear titles and labels for charts."""

# Get default templates and update only the system_prompt
# default_templates = CodeAgent.default_prompts
# prompt_templates = PromptTemplates(
#     system_prompt=system_prompt,
#     planning=default_templates.planning,
#     final_answer=default_templates.final_answer,
#     managed_agent=default_templates.managed_agent
# )

agent = CodeAgent(model = agent_model, tools = tools, add_base_tools = False)

In [7]:
prompt = "Create a linechart of the ratings according to time"

In [ ]:
result = agent.run(prompt)
print(result)